In [8]:
import pandas as pd
import numpy as np

## 표본 추출

In [3]:
# Q1
df = pd.read_csv('bike.csv')
len(df.sample(frac=0.0123))

134

In [4]:
# Q2
len(df.groupby('season').sample(frac=0.05))

545

In [7]:
# Q3
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df, train_size=0.8, random_state=123)
df_test['temp'].max()

39.36

## 데이터 전처리: 이상치, 결측치

In [50]:
df = pd.read_csv('iris_missing.csv')
df.head()

,Sepal_Length,Sepal_Width,Petal_Length,Petal_Width,Species
0,5.1,NaN,NaN,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,NaN
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,NaN


In [37]:
# Q1
df.iloc[:, :-1].isna().sum().sum()

40

In [38]:
# Q2
df.fillna(value={'Sepal_Width': df['Sepal_Width'].mean()})['Sepal_Width'].var()

0.17561945818840946

In [39]:
# Q3
sl_mean = df['Sepal_Length'].mean()
sl_std = df['Sepal_Length'].std()
len(df.loc[(df['Sepal_Length']<sl_mean-1.5*sl_std)|(df['Sepal_Length']>sl_mean+1.5*sl_std),])

19

## 데이터 전처리: 파생변수 생성

In [51]:
df = pd.read_csv('bike.csv')
df.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1


In [52]:
# Q1
(df['temp'] - df['atemp']).abs().mean()

3.5091985118501188

In [61]:
# Q2
df['datetime'] = pd.to_datetime(df['datetime'])
df['date'] = df['datetime'].dt.date
(df.groupby('date')['casual'].max() > 25).sum()

384

In [74]:
# Q3
df['hour'] = df['datetime'].dt.hour
df.groupby('hour')['registered'].mean().idxmax()

17

## 데이터 전처리: 데이터 병합

In [86]:
# Q1
df_A = pd.read_csv('join_data_group_members.csv')
df_B = pd.read_csv('join_data_member_room.csv')
pd.merge(left=df_A, right=df_B, left_on='member', right_on='name', how='left').isna().any(axis=1).sum()

5

In [103]:
# Q2
df = pd.read_csv('bike.csv')
df['datetime'] = pd.to_datetime(df['datetime'])
df['hour'] = df['datetime'].dt.hour
df_s2 = df[df['season']==2]
df_s4 = df[df['season']==4]
(df_s2.groupby('hour')['registered'].mean() - df_s4.groupby('hour')['registered'].mean()).abs().idxmax()

19

In [143]:
# Q3
df = pd.read_csv('bike.csv')
df['datetime'] = pd.to_datetime(df['datetime'])
df['date'] = df['datetime'].dt.date
df_h = df.groupby('date')['humidity'].max().reset_index()
df_h100 = df_humid[df_humid['humidity']==100]
df_join = pd.merge(left=df, right=df_h100, left_on='date', right_on='date', how='inner')
df_join[df_join['temp']>30]['count'].mean()

338.75

## 데이터 전처리: 정렬 및 변환

In [9]:
# Q1
df = pd.read_csv('bike.csv')
pd.crosstab(df['workingday'], df['holiday'], normalize=True)

holiday,0,1
workingday,,
0,0.290557,0.028569
1,0.680875,0.000000


In [42]:
# Q2
df = pd.read_csv('diamonds.csv')
pd.crosstab(df['cut'], df['color']).reset_index().melt(id_vars='cut').sort_values('value', ascending=False).head()

,cut,color,value
17,Ideal,G,4884
7,Ideal,E,3903
12,Ideal,F,3826
22,Ideal,H,3115
18,Premium,G,2924


In [44]:
# Q3
df = pd.read_csv('diamonds.csv')
df_group = df.groupby(['cut', 'color'])[['price', 'carat']].mean().reset_index()
df_group['ratio'] = df_group['price'] / df_group['carat']
df_group.sort_values('ratio', ascending=False).head()

,cut,color,price,carat,ratio
24,Premium,G,4500.742134,0.841488,5348.553755
17,Ideal,G,3720.706388,0.700715,5309.874382
23,Premium,F,4324.890176,0.827036,5229.388117
26,Premium,I,5946.180672,1.144937,5193.456761
16,Ideal,F,3374.939362,0.655829,5146.069664


## 데이터 전처리: 사용자 정의 함수 활용

In [20]:
# Q1
def udf1(x):
    return ((x**2).sum())**0.5
#    return (x.pow(2).sum())**0.5

print(udf1(pd.Series([3, 5, 9, 20])).round(2))

22.69


In [21]:
# Q2
def standardization(x):
    return (x - x.mean()) / x.std()

print(standardization(pd.Series([-4, 5, 7, 9])).round(2))

0   -1.44
1    0.13
2    0.48
3    0.83
dtype: float64


In [23]:
# Q3
def MinMax(x):
    return (x - x.min()) / (x.max() - x.min())

print(MinMax(pd.Series([-4, 5, 7, 9])).round(2))

0    0.00
1    0.69
2    0.85
3    1.00
dtype: float64
